In [1]:
import numpy as np

In [ ]:
from dataclasses import dataclass


@dataclass
class Solution:
    assignment: np.ndarray  
    total_value: int    
    load: np.ndarray         

# Ricostruisce i carichi (KxD) a partire da un assegnamento di item.
def compute_load(assignment: np.ndarray, weights: np.ndarray, K: int, D: int) -> np.ndarray:
    load = np.zeros((K, D), dtype=int)
    for i, a in enumerate(assignment):
        if a >= 0:                      
            load[a] += weights[i]
    return load

# Controlla i vincoli: nessun carico supera le capacità.
def is_feasible(load: np.ndarray, constraints: np.ndarray) -> bool:
    return np.all(load <= constraints)

# Calcola il valore totale di un assegnamento (ignora gli item fuori).
def evaluate(assignment: np.ndarray, values: np.ndarray) -> int:
    return int(values[assignment >= 0].sum())

# Inizializzazione greedy
def greedy_init(values: np.ndarray, weights: np.ndarray, constraints: np.ndarray) -> Solution:
   
    N = len(values)
    K, D = constraints.shape
    dens = values / (1.0 + weights.sum(axis=1)) 
    order = np.argsort(-dens)                  
    assignment = -np.ones(N, dtype=int)        
   
    # Carichi iniziali a zero
    load = np.zeros((K, D), dtype=int)
    total = 0

    # per ogni item prova a metterlo nello zaino dove entra e che lascia più 'slack'(spazio residuo totale dopo l'inserimento).
    for i in order:                             
        best_k, best_slack = None, None
        for k in range(K):
            # Verifica fattibilità se aggiungiamo l'item i allo zaino k
            if np.all(load[k] + weights[i] <= constraints[k]):
                slack = int((constraints[k] - (load[k] + weights[i])).sum())    # Slack = somma dello spazio residuo dopo averlo messo
                if best_slack is None or slack > best_slack:
                    best_slack, best_k = slack, k
        # Se esiste almeno uno zaino dove entra, assegnalo al migliore
        if best_k is not None:
            assignment[i] = best_k
            load[best_k] += weights[i]
            total += int(values[i])
    return Solution(assignment, total, load)

# Tenta una mossa locale su un singolo item i:
#    - new_k = -1  -> rimuovi l'item (mettilo "fuori")
#    - new_k = 0..K-1 -> sposta l'item nello zaino new_k
# Mantiene la fattibilità: se la mossa sfora la capacità, ritorna None.
def try_move(assignment, weights, values, load, constraints, i, new_k):
   
    old_k = assignment[i]
    if old_k == new_k:           # nessun cambiamento
        return None
    
    # Copia dei carichi per applicare la mossa "in prova"
    new_load = load.copy()
    if old_k >= 0:      # se era assegnato, togli il suo peso dallo zaino corrente
        new_load[old_k] -= weights[i]
    if new_k >= 0:       # se lo si vuole mettere in uno zaino, verifica fattibilità e aggiungi il peso
        if not np.all(new_load[new_k] + weights[i] <= constraints[new_k]):
            return None
        new_load[new_k] += weights[i]

    # Aggiorna assegnamento e valore totale
    new_assignment = assignment.copy()
    new_assignment[i] = new_k
    new_value = evaluate(new_assignment, values)
    return new_assignment, new_load, new_value


# Simulated Annealing:
#     - parte dalla soluzione greedy
#     - ad ogni passo sceglie un item a caso e un 'contenitore' a caso {-1, 0..K-1}
#     - se la mossa è fattibile:
#         * accetta sempre se migliora (delta >= 0)
#         * se peggiora, accetta con probabilità exp(delta / T)
#     - T si raffredda moltiplicandola per 'cooling' < 1.
#     Restituisce il miglior risultato visto (best-so-far).
def simulated_annealing(values: np.ndarray, weights: np.ndarray, constraints: np.ndarray,
                        steps: int = 40_000, temp0: float = 2.0, cooling: float = 0.9997,
                        seed: int = 123) -> Solution:
    
    rng = np.random.default_rng(seed)
    sol = greedy_init(values, weights, constraints)      # Punto di partenza: soluzione greedy
    best = Solution(sol.assignment.copy(), sol.total_value, sol.load.copy())
    temp = temp0
    N = len(values); K, D = constraints.shape

    for _ in range(steps):
        i = int(rng.integers(0, N)) # item scelto a caso
        # new_k uniforme in {-1, 0, 1, ..., K-1}
        u = int(rng.integers(0, K + 1))   # conenitore scelto a caso in modo uniforme su {-1, 0, ..., K-1} 
        new_k = -1 if u == 0 else (u - 1)

        # tenta la mossa
        trial = try_move(sol.assignment, weights, values, sol.load, constraints, i, new_k)
        if trial is not None:
            new_assignment, new_load, new_value = trial
            delta = new_value - sol.total_value
            # accetta miglioramenti; peggioramenti solo con probabilità exp(delta/temp)
            if delta >= 0 or rng.random() < np.exp(delta / max(1e-9, temp)):
                sol = Solution(new_assignment, new_value, new_load)
                if sol.total_value > best.total_value:  # aggiorn ail best se migliora
                    best = Solution(sol.assignment.copy(), sol.total_value, sol.load.copy())
        temp *= cooling
    return best


TEST PROBLEMS

In [6]:
# =========================
# Inizializzazione (Problem 1)
# =========================
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

best = simulated_annealing(VALUES, WEIGHTS, CONSTRAINTS,
                           steps=40_000, temp0=2.0, cooling=0.9997, seed=123)
print("=== RISULTATO PROBLEMA 1===")
print("Valore totale:", best.total_value)


# =========================
# Inizializzazione (Problem 2)
# =========================
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

best = simulated_annealing(VALUES, WEIGHTS, CONSTRAINTS,
                           steps=40_000, temp0=2.0, cooling=0.9997, seed=123)
print("=== RISULTATO PROBLEMA 2===")
print("Valore totale:", best.total_value)


# =========================
# Inizializzazione (Problem 3)
# =========================
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

best = simulated_annealing(VALUES, WEIGHTS, CONSTRAINTS,
                           steps=40_000, temp0=2.0, cooling=0.9997, seed=123)
print("=== RISULTATO PROBLEMA 3===")
print("Valore totale:", best.total_value)

=== RISULTATO PROBLEMA 1===
Valore totale: 1065
=== RISULTATO PROBLEMA 2===
Valore totale: 43952
=== RISULTATO PROBLEMA 3===
Valore totale: 1821466
